In [5]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])

s1 = "hello"
b1 = s1.encode("utf-8")
b2 = b'\x81\x81'
# print(b1)
b2.decode("utf-8")
# print(decode_utf8_bytes_to_str_wrong(b2))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x81 in position 0: invalid start byte

In [ ]:
import numpy as np
import tqdm
txt_path = "data/TinyStoriesV2-GPT4-train.txt"   # 里面是用空格/换行分隔的整数token或浮点数
bin_path = "data/TinyStoriesV2-GPT4-train.uint16.bin"  # 目标二进制文件
dtype = np.uint16  # 或 np.float32，务必和你的真实数据一致！

# 方法1：若文本里是整数/浮点的 ASCII，且内存足够一次读（中小数据）
# arr = np.loadtxt(txt_path, dtype=dtype)   # 慢但简单
# arr.tofile(bin_path)

# 方法2：流式转换，适合巨大文本（节省内存）
count = 0
with open(txt_path, "r", encoding="utf-8") as fin, open(bin_path, "wb") as fout:
    buf = []
    CHUNK = 1_000_000  # 每批次转换这么多数字
    for line in fin:
        # 将一行里的 token 解析为整数/浮点；如果是空格分隔，split() 即可
        if line.strip():
            buf.extend(line.split())
        # 分批写出
        if len(buf) >= CHUNK:
            np.asarray(buf[:CHUNK], dtype=dtype).tofile(fout)
            del buf[:CHUNK]
            count += CHUNK
    # 写出余下的
    if buf:
        np.asarray(buf, dtype=dtype).tofile(fout)
        count += len(buf)

print("wrote", count, "elements to", bin_path)

ValueError: invalid literal for int() with base 10: 'Once'